In [5]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta

# Constants
API_KEY = '' #add the API
SINCE_DATE = int(datetime(2023, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', window=90, normalize=True):
    # Calculate price changes
    price_change = df[column].diff()

    # Calculate gains and losses
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)

    # Calculate average gains and losses
    avg_gains = gains.rolling(window=window, min_periods=1).mean()
    avg_losses = losses.rolling(window=window, min_periods=1).mean()

    # Calculate relative strength
    relative_strength = avg_gains / avg_losses

    # Calculate RSI
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        # Normalize RSI to -1 to +1 range
        rsi_min = rsi.rolling(window=window, min_periods=1).min()
        rsi_max = rsi.rolling(window=window, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

# Apply the function to our merged_df, specifically for price_usd_close with a 90-day window
merged_df['Price Momentum (Normalized)'] = calculate_momentum_rsi(merged_df, column='price_usd_close', window=90, normalize=True)
merged_df['Price Momentum (RSI)'] = calculate_momentum_rsi(merged_df, column='price_usd_close', window=90, normalize=False)

# Display the first few rows to verify
(merged_df.head())

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum (Normalized),Price Momentum (RSI)
t,,,,,
2023-01-01,16620.819927,6.198049e+06,3.123859e+09,NaN,NaN
2023-01-02,16693.930009,1.863979e+07,4.123114e+09,NaN,100.000000
2023-01-03,16682.715885,-8.071869e+06,4.959870e+09,-1.000000,86.701181
2023-01-04,16865.159755,2.188922e+07,7.058124e+09,0.367809,95.796302
2023-01-05,16841.821596,-1.490683e+07,4.820176e+09,-0.791169,88.089783
